In [2]:
import sys
sys.path.insert(0, "../src")  # Ensure your local src folder is first in the search path
from my_llm.my_utils import clean_text_for_bert
from my_llm.my_utils import convert_insta_txt_to_jsonl
from my_llm.my_utils import remove_empty_body_entries
from my_llm.my_utils import add_form_field
from my_llm.my_utils import add_company_voice
from my_llm.my_utils import load_forms_config
from my_llm.my_utils import enrich_email_with_contact_response
from my_llm.my_utils import process_emails
from my_llm.my_utils import load_socials_config
from my_llm.my_utils import enrich_instagram_post
from my_llm.my_utils import process_instagram_posts
from my_llm.my_utils import enrich_website_content
from my_llm.my_utils import process_website_posts
from my_llm.my_utils import parse_brandbook
from my_llm.my_utils import convert_brandbook_file
from my_llm.my_utils import rework_emails_to_prompt_completion
from my_llm.my_utils import convert_documents_clean_to_jsonl
from my_llm.my_utils import load_all_jsonl_files
from my_llm.my_utils import load_normalized_jsonl
from my_llm.my_utils import reformat_prompt
from my_llm.my_utils import clean_dataframe
from my_llm.my_utils import filter_friendly_rows
from my_llm.my_utils import filter_out_nan
from my_llm.my_utils import save_df_to_jsonl
from my_llm.my_utils import remove_curly_braces

import pandas as pd 
import yaml
import re
import json

In [3]:
# Define the path to your YAML configuration file
config_path = "/Users/blairjdaniel/AI-Assistant-Springs/config/data_config.yaml"

# Load the YAML configuration
with open(config_path, "r") as file:
    config = yaml.safe_load(file)

# Print the loaded configuration to verify its contents
print(config)

{'data': {'source': '/Users/blairjdaniel/AI-Assistant-Springs/data', 'cleaned_data_path': '/Users/blairjdaniel/AI-Assistant-Springs/data/outputs', 'split_ratio': 0.8}, 'preprocessing': {'remove_duplicates': True, 'text_cleaning': {'lower_case': False, 'remove_punctuation': False, 'remove_stopwords': False}}}


In [4]:
# Define the path to your YAML configuration file
config_path = "/Users/blairjdaniel/AI-Assistant-Springs/config/baseline_template.yaml"

# Load the YAML configuration
with open(config_path, "r") as file:
    config = yaml.safe_load(file)

# Print the loaded configuration to verify its contents
print(config)

{'forms': {'contact': {'subject': 'Thanks for reaching out to Springs RV Resort', 'body': '<html>\n  <body>\n    <p>Hi {first_name},</p>\n    <p>Thanks so much for reaching out! My name is Kelsey, and I’m the Sales Assistant here at Springs RV Resort.</p>\n    <p>I’d be happy to set you up with our Sales Manager, Jamie Smith, for a personal resort tour so you can get a feel for what makes the Springs so special. You’re welcome to reply with a day and time that works best for you, or you can use our online calendar to book directly: <a href="https://calendly.com/springs-rv-resort/springs-rv-resort-sales-meeting">BOOK HERE</a> or call 778-871-3160.</p>\n    <p>As you may know, our resort offers daily rentals, seasonal rentals, and RV lot ownership. If you’re exploring ownership, our brand-new Phase 3 lots range from $269,000 to over $300,000. We also have a selection of resale lots available; you can browse listings with photos and pricing <a href="https://springsrv.com/resale-lots-for-s

In [5]:
# Define the path to your YAML configuration file
config_path = "/Users/blairjdaniel/AI-Assistant-Springs/config/baseline_responses.yaml"

# Load the YAML configuration
with open(config_path, "r") as file:
    config = yaml.safe_load(file)

# Print the loaded configuration to verify its contents
print(config)

{'forms': {'contact': {'subject': 'Thanks for reaching out to Springs RV Resort', 'body': 'Hi {first_name},\n\nThanks so much for reaching out! My name is Kelsey, and I’m the Sales Assistant here at Springs RV Resort.\n\nI’d be happy to set you up with our Sales Manager, Jamie Smith, for a personal resort tour so you can get a feel for what makes the Springs so special. You’re welcome to reply with a day and time that works best for you, or you can book directly using our online calendar (BOOK HERE) or call 778-871-3160.\n\nAs you may know, our resort offers daily rentals, seasonal rentals, and RV lot ownership. If you’re exploring ownership, our brand-new Phase 3 lots range from $269,000 to over $300,000. We also have a selection of resale lots available; you can browse listings with photos and pricing (see details here).\n\nIf you have any questions or would like to chat before your visit, I’m always here to help!\n\nWarmly,\nKelsey\nSales Assistant\nsales@springsrv.com\n\nPlease not

In [5]:
# # Load text from a file
# with open("/Users/blairjdaniel/AI-Assistant-Springs/data/cache/documents/brandbook.txt", "r") as f:
#     text = f.read()

# # Retrieve cleaning settings from the YAML (inside data_config.yaml -> preprocessing -> text_cleaning)
# cleaning_conf = config.get("preprocessing", {}).get("text_cleaning", {})

# # Clean the text using your helper function
# cleaned_text = clean_text_for_bert(text, cleaning_conf)

# # Save cleaned text to a new file (plain text)
# with open("/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/brandbook_clean.txt", "w") as f:
#     f.write(cleaned_text)

In [6]:
# import json

# input_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/brandbook_clean.txt"
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/brandbook.jsonl"

# with open(input_path, "r") as infile:
#     text = infile.read()

# parsed_entries = parse_brandbook(text)

# with open(output_path, "w") as outfile:
#     json.dump(parsed_entries, outfile, indent=4)

# print("Parsed brand book saved to", output_path)

In [7]:
# input_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/brandbook.jsonl"
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/brandbook_converted.jsonl"
# convert_brandbook_file(input_path, output_path, default_prompt="Brandbook entry:")

In [8]:
# # Load text from a file
# with open("/Users/blairjdaniel/AI-Assistant-Springs/data/cache/documents/documents.txt", "r") as f:
#     text = f.read()

# # Retrieve cleaning settings from the YAML (inside data_config.yaml -> preprocessing -> text_cleaning)
# cleaning_conf = config.get("preprocessing", {}).get("text_cleaning", {})

# # Clean the text using your helper function
# cleaned_text = clean_text_for_bert(text, cleaning_conf)

# # Save cleaned text to a new file (plain text)
# with open("/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/documents_clean.txt", "w") as f:
#     f.write(cleaned_text)

In [9]:
# input_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/cache/insta/insta.txt"
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/instagram.jsonl"
# convert_insta_txt_to_jsonl(input_path, output_path)

In [10]:
# # Load text from a file
# with open("/Users/blairjdaniel/AI-Assistant-Springs/data/cache/website/website.txt", "r") as f:
#     text = f.read()

# # Retrieve cleaning settings from the YAML (inside data_config.yaml -> preprocessing -> text_cleaning)
# cleaning_conf = config.get("preprocessing", {}).get("text_cleaning", {})

# # Clean the text using your helper function
# cleaned_text = clean_text_for_bert(text, cleaning_conf)

# # Save cleaned text to a new file (plain text)
# with open("/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/website_clean.txt", "w") as f:
#     f.write(cleaned_text)

In [11]:
# input_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/cache/website/website.txt"
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/website.jsonl"
# convert_insta_txt_to_jsonl(input_path, output_path)

In [12]:
# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/cache/json/email_catcher/emails_all.json"
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/emails_all.json"
# remove_empty_body_entries(input_file, output_file)
# print(f"Filtered output written to {output_file}")

In [13]:
# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/cache/json/email_catcher/emails.json"
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/emails_random.json"
# remove_empty_body_entries(input_file, output_file)
# print(f"Filtered output written to {output_file}")

In [14]:
#add_form_field("/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/emails_random.json", "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/emails_random.json", "random")

In [15]:
# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/emails_phase3.json"
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/emails_phase3.jsonl"
# voice = "Friendly, professional, and warm - reflecting Springs RV Resort's brand identity."
# add_company_voice(input_file, output_file, voice)
# print(f"Updated file with company_voice metadata written to {output_file}")

In [16]:
# import json

# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/emails_phase3.jsonl"  # if it's a JSON array
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/emails_phase3.jsonl"

# import os

# if not os.path.exists(input_file):
#     raise FileNotFoundError(f"The file {input_file} does not exist. Please check the file path.")

# with open(input_file, "r") as infile:
#     data = json.load(infile)  # expects an array of objects

# with open(output_file, "w") as outfile:
#     for entry in data:
#         json_line = json.dumps(entry)
#         outfile.write(json_line + "\n")

In [17]:
# emails_jsonl = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/emails_pricelist.jsonl"
# forms_yaml = "/Users/blairjdaniel/AI-Assistant-Springs/config/baseline_template.yaml"  # Your YAML file with baseline responses
# output_jsonl = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_pricelist.jsonl"
# process_emails(emails_jsonl, forms_yaml, output_jsonl)
# print("Enriched email data written to", output_jsonl)

In [18]:
# instagram_jsonl = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/instagram.jsonl"
# socials_yaml = "/Users/blairjdaniel/AI-Assistant-Springs/config/socials_response.yaml"
# output_jsonl = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/instagram_enriched.jsonl"
# process_instagram_posts(instagram_jsonl, socials_yaml, output_jsonl)
# print("Enriched Instagram data written to", output_jsonl)

In [19]:
# instagram_jsonl = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/website.jsonl"
# socials_yaml = "/Users/blairjdaniel/AI-Assistant-Springs/config/socials_response.yaml"
# output_jsonl = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/website_enriched.jsonl"
# process_instagram_posts(instagram_jsonl, socials_yaml, output_jsonl)
# print("Enriched Instagram data written to", output_jsonl)

In [20]:
# website_jsonl = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/website.jsonl"
# socials_yaml = "/Users/blairjdaniel/AI-Assistant-Springs/config/socials_response.yaml"
# output_jsonl = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/website_enriched.jsonl"
# process_website_posts(website_jsonl, socials_yaml, output_jsonl)
# print("Enriched website data written to", output_jsonl)

In [ ]:
# input_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/documents_final.jsonl"
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/training_data.jsonl"
# with open(input_path, "r") as infile:
#     conversations = json.load(infile)

# with open(output_path, "w") as outfile:
#     for conversation in conversations:
#         example = convert_conversation_to_example(conversation)
#         outfile.write(json.dumps(example) + "\n")

# print("Training file created at", output_path)

In [22]:
# input_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/emails_contact.json"
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/emails_contact_converted.jsonl"
# rework_emails_to_prompt_completion(input_path, output_path)

In [23]:
# input_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/emails_inquiries.json"
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/emails_inquiries_converted.jsonl"
# rework_emails_to_prompt_completion(input_path, output_path)

In [24]:
# input_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/emails_waitlist.json"
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/emails_waitlist_converted.jsonl"
# rework_emails_to_prompt_completion(input_path, output_path)

In [25]:
# input_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/emails_pricelist.json"
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/emails_pricelist_converted.jsonl"
# rework_emails_to_prompt_completion(input_path, output_path)

In [26]:
# # Load text from a file
# with open("/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/documents_clean.txt", "r") as f:
#     text = f.read()

# # Retrieve cleaning settings from the YAML (inside data_config.yaml -> preprocessing -> text_cleaning)
# cleaning_conf = config.get("preprocessing", {}).get("text_cleaning", {})

# # Clean the text using your helper function
# cleaned_text = clean_text_for_bert(text, cleaning_conf)

# # Save cleaned text to a new file (plain text)
# with open("/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/documents_clean.txt", "w") as f:
#     f.write(cleaned_text)

In [27]:
# # Ensure the function is imported
# from my_llm.my_utils import convert_documents_clean_to_jsonl

# # Use the function to convert the documents
# input_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/documents_clean.txt"
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/documents_clean_converted.jsonl"
# convert_documents_clean_to_jsonl(input_path, output_path)

In [28]:
# data_dir = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab"
# combined_data = load_all_jsonl_files(data_dir)
# print("Combined dataset count:", len(combined_data))

In [29]:
# # Write the combined_data list to a temporary JSONL file
# import tempfile
# import json

# with tempfile.NamedTemporaryFile(mode='w+', suffix='.jsonl', delete=False) as temp_file:
# 	for entry in combined_data:
# 		temp_file.write(json.dumps(entry) + '\n')
# 	temp_file_path = temp_file.name

# # Load the normalized data from the temporary JSONL file
# normalized_combined_data = load_normalized_jsonl(temp_file_path)
# print("Normalized DataFrame shape:", normalized_combined_data.shape)

In [30]:
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/data.jsonl"
# normalized_combined_data.to_json(output_path, orient='records', lines=True)
# print(f"Saved normalized data to {output_path}")

In [ ]:
# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/emails_contact_converted.jsonl"
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_data/data_cleaned/emails_contact_converted_clean.jsonl"

# with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
#     for line in infile:
#         record = json.loads(line)
#         if 'prompt' in record:
#             record['prompt'] = reformat_prompt(record['prompt'])
#         outfile.write(json.dumps(record) + "\n")

# print("Reformatted JSONL saved to:", output_file)

Reformatted JSONL saved to: /Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_data/data_cleaned/emails_contact_converted_clean.jsonl


In [ ]:
# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/emails_inquiries_converted.jsonl"
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_inquiries_final.jsonl"

# with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
#     for line in infile:
#         record = json.loads(line)
#         if 'prompt' in record:
#             record['prompt'] = reformat_prompt(record['prompt'])
#         outfile.write(json.dumps(record) + "\n")

# print("Reformatted JSONL saved to:", output_file)

In [ ]:
# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_pricelist_converted.jsonl"
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_data/email_data_cleaned/emails_pricelist_converted_clean.jsonl"

# with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
#     for line in infile:
#         record = json.loads(line)
#         if 'prompt' in record:
#             record['prompt'] = reformat_prompt(record['prompt'])
#         outfile.write(json.dumps(record) + "\n")

# print("Reformatted JSONL saved to:", output_file)

In [ ]:
# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_waitlist_converted.jsonl"
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_data/email_data_cleaned/emails_waitlist_converted_clean.jsonl"

# with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
#     for line in infile:
#         record = json.loads(line)
#         if 'prompt' in record:
#             record['prompt'] = reformat_prompt(record['prompt'])
#         outfile.write(json.dumps(record) + "\n")

# print("Reformatted JSONL saved to:", output_file)

In [35]:
# # Load all json file in a dir
# data_dir = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_data/data_cleaned"
# combined_data = load_all_jsonl_files(data_dir)
# print("Combined dataset count:", len(combined_data))

In [36]:
# import pandas as pd

# df = pd.DataFrame(combined_data)
# df.to_json('/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_data/data_cleaned/data.json', orient='records', lines=True)

In [ ]:
# df = pd.read_parquet("hf://datasets/Victorano/customer-support-training-dataset/data/train-00000-of-00001.parquet")
# df

In [ ]:
# # If you need to reformat each prompt, apply your reformat_prompt function:
# #df["prompt"] = df["instruction"].apply(reformat_prompt)
# #df["completion"] = df["response"].apply(reformat_prompt)
# df = df.rename(columns={'instruction': 'prompt', 'response': 'completion'})
# df = df.drop(columns=['flags'])

In [ ]:
# # Example usage:
# friendly_words = [
#     "Thank You", "understandable", "Definitely", "Absolutely", "Certainly",
#     "Exactly", "Completely", "Quickly", "Fantastic", "Great", "Marvellous",
#     "Excellent", "Enjoy", "Splendid", "Essential", "Generous", "Recommend",
#     "Friendly", "Impressive", "Interesting", "Brilliant", "Exciting", "Terrific",
#     "Fascinating", "Expert", "Favourite", "Ideal"
# ]

# filtered_data = filter_friendly_rows(df, friendly_words, columns=['prompt','completion'])


In [ ]:
# df = filtered_data


In [ ]:
# synthetic_data = pd.read_json('/Users/blairjdaniel/AI-Assistant-Springs/data/cache/synthetic_data/synthetic_data.jsonl', lines=True)
# synthetic_data['category'] = 'contact'
# synthetic_data['intent'] = 'contact'
# synthetic_data

In [ ]:
# df = pd.concat([df, synthetic_data], ignore_index=True)
# df = df.drop_duplicates()
# # Verify the number of rows
# print(f"Number of rows after removing duplicates: {len(df)}")

Number of rows after removing duplicates: 14135


In [ ]:
# filtered_df = filter_out_nan(df, columns=['prompt', 'completion'])
# filtered_df

In [ ]:
# filtered_df = filtered_df[~filtered_df['intent'].isin(['delivery_options', 'track_order', 'track_refund', 'change_shipping_address', 'cancel_order'])]

In [ ]:
# # Assuming `filtered_df` is your DataFrame
# columns_to_clean = ["prompt", "completion"]  # Specify the columns to clean
# filtered_df = clean_dataframe(filtered_df, columns_to_clean)

In [ ]:
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/attitude_data.jsonl"
# save_df_to_jsonl(filtered_df, output_path)

DataFrame successfully saved to /Users/blairjdaniel/AI-Assistant-Springs/data/outputs/attitude_data.jsonl


In [ ]:
# def remove_curly_braces(text):
#     """
#     Removes all curly braces `{}` and their contents from the given text.
#     """
#     if isinstance(text, str):  # Ensure the input is a string
#         return re.sub(r"\{\{.*?\}\}", "", text).strip()
#     return text  # Return unchanged if not a string

# # Apply the function to specific columns in the DataFrame
# columns_to_clean = ["prompt", "completion"]  # Specify the columns to clean
# for column in columns_to_clean:
#     filtered_df[column] = filtered_df[column].apply(remove_curly_braces)

# # Save the cleaned DataFrame back to a JSONL file
# output_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/cleaned_attitude_data.jsonl"
# filtered_df.to_json(output_path, orient="records", lines=True)

In [ ]:
# import json

# def convert_chat_to_jsonl(input_file, output_file):
#     """
#     Converts chat data from a text file into JSONL format with Human 1 as 'prompt',
#     Human 2 as 'completion', and adds 'category' and 'intent' fields.

# #     :param input_file: Path to the input text file containing chat data.
# #     :param output_file: Path to save the converted JSONL file.
#     """
#     with open(input_file, "r") as infile, open(output_file, "w") as outfile:
#         current_prompt = None
#         current_completion = None

#         for line in infile:
#             line = line.strip()
#             if line.startswith("question:"):
#                 # Save the current prompt
#                 current_prompt = line.replace("question:", "").strip()
#             elif line.startswith("answer:") and current_prompt:
#                 # Save the current completion
#                 current_completion = line.replace("answer:", "").strip()

#                 # Write the JSONL entry
#                 if current_prompt and current_completion:
#                     entry = {
#                         "prompt": current_prompt,
#                         "completion": current_completion,
#                         "category": "conversation",
#                         "intent": "friendly_conversation"
#                     }
#                     outfile.write(json.dumps(entry) + "\n")

#                 # Reset for the next conversation
#                 current_prompt = None
#                 current_completion = None

# # Example usage
# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/cache/general/Conversation.csv"
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/human_chat_1_final.jsonl"
# convert_chat_to_jsonl(input_file, output_file)

In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/human_chat_final.jsonl"
# chat_df = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", chat_df.shape)
# chat_df

In [ ]:
# def remove_curly_braces(text):
#     """
#     Removes all curly braces `{}` and their contents from the given text.
#     """
#     if isinstance(text, str):  # Ensure the input is a string
#         return re.sub(r"\{\{.*?\}\}", "", text).strip()
#     return text  # Return unchanged if not a string

# # Apply the function to specific columns in the DataFrame
# columns_to_clean = ["prompt", "completion"]  # Specify the columns to clean
# for column in columns_to_clean:
#     chat_df[column] = chat_df[column].apply(remove_curly_braces)

In [ ]:
# df = pd.concat([filtered_df, chat_df], ignore_index=True)
# df = df.drop_duplicates()
# # Verify the number of rows
# print(f"Number of rows after removing duplicates: {len(df)}")

Number of rows after removing duplicates: 12699


In [ ]:
# import csv
# import json

# # Input and output file paths
# input_csv_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/cache/general/Conversation.csv"
# output_jsonl_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/human_chat_1.jsonl"

# # Open the CSV file and read its contents
# with open(input_csv_path, mode="r", encoding="utf-8") as csv_file:
#     csv_reader = csv.DictReader(csv_file)
    
#     # Open the JSONL file for writing
#     with open(output_jsonl_path, mode="w", encoding="utf-8") as jsonl_file:
#         for row in csv_reader:
#             # Create the JSON object for each row
#             json_object = {
#                 "prompt": row["question"],
#                 "completion": row["answer"],
#                 "category": "conversation",
#                 "intent": "conversation"
#             }
#             # Write the JSON object as a line in the JSONL file
#             jsonl_file.write(json.dumps(json_object) + "\n")

# print(f"Conversion complete. JSONL file saved to {output_jsonl_path}")

Conversion complete. JSONL file saved to /Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/human_chat_1.jsonl


In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/human_chat_1.jsonl"
# chat1_df = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", chat1_df.shape)
# chat1_df

In [ ]:
# def remove_curly_braces(text):
#     """
#     Removes all curly braces `{}` and their contents from the given text.
#     """
#     if isinstance(text, str):  # Ensure the input is a string
#         return re.sub(r"\{\{.*?\}\}", "", text).strip()
#     return text  # Return unchanged if not a string

# # Apply the function to specific columns in the DataFrame
# columns_to_clean = ["prompt", "completion"]  # Specify the columns to clean
# for column in columns_to_clean:
#     chat1_df[column] = chat1_df[column].apply(remove_curly_braces)

In [ ]:
# df = pd.concat([df, chat1_df], ignore_index=True)
# df = df.drop_duplicates()
# # Verify the number of rows
# print(f"Number of rows after removing duplicates: {len(df)}")

In [ ]:
# #Define the output JSONL file path
# output_jsonl_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/human_chat.jsonl"

# # Save the DataFrame to a JSONL file
# df.to_json(output_jsonl_path, orient='records', lines=True)

# print(f"DataFrame saved to JSONL file at {output_jsonl_path}")

DataFrame saved to JSONL file at /Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/human_chat.jsonl


In [ ]:
# df = pd.read_json('/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/human_chat.jsonl', lines = True)
# emotion = pd.read_csv('/Users/blairjdaniel/AI-Assistant-Springs/data/cache/general/emotion-emotion_69k.csv')
# df.head()

In [ ]:
# # Assuming `df` is your DataFrame
# df['category'] = 'conversation_customer_service'
# df.head()

In [ ]:
# import csv
# import json

# # Input and output file paths
# input_csv_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/cache/general/emotion-emotion_69k.csv"
# output_jsonl_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emotion.jsonl"

# # Open the CSV file and read its contents
# with open(input_csv_path, mode="r", encoding="utf-8") as csv_file:
#     csv_reader = csv.DictReader(csv_file)
    
#     # Open the JSONL file for writing
#     with open(output_jsonl_path, mode="w", encoding="utf-8") as jsonl_file:
#         for row in csv_reader:
#             # Create the JSON object for each row
#             json_object = {
#                 "prompt": row["Situation"],
#                 "completion": row["empathetic_dialogues"],
#                 "category": "conversation_normal",
#                 "intent": "conversation_emphatetic"
#             }
#             # Write the JSON object as a line in the JSONL file
#             jsonl_file.write(json.dumps(json_object) + "\n")

# print(f"Conversion complete. JSONL file saved to {output_jsonl_path}")

Conversion complete. JSONL file saved to /Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emotion.jsonl


In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emotion.jsonl"
# emotion_df = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", emotion_df.shape)
# emotion_df.drop_duplicates()
# emotion_df

In [ ]:
# df_all = pd.concat([df, emotion_df], ignore_index=True)
# df_all = df_all.drop_duplicates()
# # Verify the number of rows
# print(f"Number of rows after removing duplicates: {len(df_all)}")

Number of rows after removing duplicates: 80764


In [ ]:
# #Define the output JSONL file path
# output_jsonl_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emotion_human.jsonl"

# # Save the DataFrame to a JSONL file
# df_all.to_json(output_jsonl_path, orient='records', lines=True)

# print(f"DataFrame saved to JSONL file at {output_jsonl_path}")

DataFrame saved to JSONL file at /Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emotion_human.jsonl


In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/documents_final.jsonl"
# documents_df = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", documents_df.shape)
# documents_df = documents_df.drop(columns=['category', 'intent'])


In [ ]:
# import json

# # File paths
# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/website_final.jsonl"
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/website_with_intent.jsonl"

# # Add intent column to each record
# with open(input_file, "r") as infile, open(output_file, "w") as outfile:
#     for line in infile:
#         record = json.loads(line)
#         record["intent"] = "create website post"  # Add the intent field
#         outfile.write(json.dumps(record) + "\n")

# print(f"Updated file with intent column saved to {output_file}")



Updated file with intent column saved to /Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/website_with_intent.jsonl


In [ ]:
# import json

# # File paths
# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_inquiries_final.jsonl"
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_inquiries_with_intent.jsonl"

# # Add intent column to each record
# with open(input_file, "r") as infile, open(output_file, "w") as outfile:
#     for line in infile:
#         record = json.loads(line)
#         record["intent"] = "general inquiries"  # Add the intent field
#         outfile.write(json.dumps(record) + "\n")

# print(f"Updated file with intent column saved to {output_file}")

Updated file with intent column saved to /Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_inquiries_with_intent.jsonl


In [ ]:
# import json

# # File paths
# input_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_waitlist_final.jsonl"
# output_file = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_waitlist_with_intent.jsonl"

# # Add intent column to each record
# with open(input_file, "r") as infile, open(output_file, "w") as outfile:
#     for line in infile:
#         record = json.loads(line)
#         record["intent"] = "waitlist"  # Add the intent field
#         outfile.write(json.dumps(record) + "\n")

# print(f"Updated file with intent column saved to {output_file}")

Updated file with intent column saved to /Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_waitlist_with_intent.jsonl


In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_contact_with_intent.jsonl"
# email_contact_df = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", email_contact_df.shape)

# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_inquiries_with_intent.jsonl"
# email_inquiries_df = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", email_inquiries_df.shape)

# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails_waitlist_with_intent.jsonl"
# email_waitlist_df = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", email_waitlist_df.shape)


Normalized DataFrame shape: (68, 3)
Normalized DataFrame shape: (68, 3)
Normalized DataFrame shape: (24, 3)


In [ ]:
# df = pd.concat([email_contact_df, email_inquiries_df, email_waitlist_df], ignore_index=True)
# df = df.drop_duplicates()
# # Verify the number of rows
# print(f"Number of rows after removing duplicates: {len(df)}")

Number of rows after removing duplicates: 152


In [ ]:
# #Define the output JSONL file path
# output_jsonl_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails.jsonl"

# # Save the DataFrame to a JSONL file
# df.to_json(output_jsonl_path, orient='records', lines=True)

# print(f"DataFrame saved to JSONL file at {output_jsonl_path}")

DataFrame saved to JSONL file at /Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/emails.jsonl


In [ ]:
#  # Load all json file in a dir
# data_dir = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/finals"
# combined_data = load_all_jsonl_files(data_dir)
# print("Combined dataset count:", len(combined_data))

Combined dataset count: 314


In [ ]:
# data = pd.DataFrame(combined_data)
# data = data.drop(columns=['category'])
# data

,prompt,completion,intent
0,Lot 168 is the lot I’m interested in but would...,"Hi, Thanks so much for reaching out! My name i...",contact us
1,Firstly renting and then possibly buying a pad...,"Hi, Thanks so much for reaching out! My name i...",contact us
2,A tour of the resort and activities before we ...,"Hi, Thanks so much for reaching out! My name i...",contact us
3,Private RV\nQuestions: Pricing I saw the $210 ...,"Hi, Thanks so much for reaching out! My name i...",contact us
4,Rv lot\nQuestions:,"Hi, Thanks so much for reaching out! My name i...",contact us
...,...,...,...
309,Create a website entry about contact info at t...,CONTACT US Email: contact@springsrv.com Office...,create website post
310,Create a website entry about phase 3 lots for ...,Phase 3 New Lots Now Available We are excited ...,create website post
311,Create a website entry about phase 3 pricelist...,View Phase 3 Price List A SUMMER HAVEN SO CLOS...,create website post
312,Create a website entry about tips at the Sprin...,Why not have a vacation on your doorstep all y...,create website post


In [ ]:
# #Define the output JSONL file path
# output_jsonl_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/finals/final_data.jsonl"

# # Save the DataFrame to a JSONL file
# data.to_json(output_jsonl_path, orient='records', lines=True)

# print(f"DataFrame saved to JSONL file at {output_jsonl_path}")

DataFrame saved to JSONL file at /Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/finals/final_data.jsonl


In [14]:
 # Load all json file in a dir
data_dir = "/Users/blairjdaniel/AI-Assistant-Springs/data/pre-output/email"
data = load_all_jsonl_files(data_dir)
print("Combined dataset count:", len(data))

Combined dataset count: 552


In [16]:
data = pd.DataFrame(combined_data)
#data = data.drop(columns=['category'])
data = data.drop(columns=['from', 'date', 'body', 'form',	'company_voice'])

In [19]:
data['label'] = "contact"
data.to_csv('/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/finals/final_email.csv')

In [ ]:
import pandas as pd

# Sample data for demonstration
data = {
    "subject": [
        "Springs RV Resort Contact Us",
        "Springs RV Resort Join the waitlist",
        "RV Lot Phase 3 Price List - Springs RV Resort",
         "RV Lot Sales - Springs RV Resort",
        "I have a question about your services",
        "Interested in booking a tour",
        "Need more info on rentals"
    ]
}
df = pd.DataFrame(data)

# Step 1: Define a function to assign a textual label based on the subject
def assign_label(subject):
    if subject == "Springs RV Resort Contact Us":
        return "contact"
    elif subject == "Springs RV Resort Join the waitlist":
        return "waitlist"
    elif subject == "RV Lot Phase 3 Price List - Springs RV Resort":
        return "pricelist"
    else:
        return "inquiries"

# Apply the function to create a new column for textual labels
df["label_text"] = df["subject"].apply(assign_label)

# Step 2: Map the textual labels to numeric values
label_mapping = {"contact": 0, "waitlist": 1, "pricelist": 2, "inquiries": 3}
df["label"] = df["label_text"].map(label_mapping)

print(df)

data = pd.read_csv('/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/finals/final_email.csv')

# Step 1: Define a function to assign a textual label based on the subject
def assign_label(subject):
    if subject == "Springs RV Resort Contact Us":
        return "contact"
    elif subject == "Springs RV Resort Join the waitlist":
        return "waitlist"
    elif subject == "RV Lot Phase 3 Price List - Springs RV Resort":
        return "pricelist"
    else:
        return "inquiries"

# Apply the function to create a new column for textual labels
data["label_text"] = data["subject"].apply(assign_label)

# Step 2: Map the textual labels to numeric values
label_mapping = {"contact": 0, "waitlist": 1, "pricelist": 2, "inquiries": 3}
data["label"] = data["label_text"].map(label_mapping)

print(data)

In [29]:
data = pd.concat([data, df], ignore_index=True)
data.head(50)
#data = data.drop(columns=['Unnamed: 0'])


,subject,label,label_text
0,Springs RV Resort Contact Us,0,contact
1,Springs RV Resort Contact Us,0,contact
2,Springs RV Resort Contact Us,0,contact
3,Springs RV Resort Contact Us,0,contact
4,Springs RV Resort Contact Us,0,contact
5,Springs RV Resort Contact Us,0,contact
6,Springs RV Resort Contact Us,0,contact
7,Springs RV Resort Contact Us,0,contact
8,Springs RV Resort Contact Us,0,contact
9,Springs RV Resort Contact Us,0,contact


In [26]:
data.to_csv('/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/finals_all.csv')